### **Installing and importing required dependencies**

In [1]:
!pip install boto3 sagemaker sciket-learn

In [1]:
import sagemaker
import boto3
import pandas as pd
import numpy as np
import os
import logging
from datetime import datetime
from sagemaker.session import TrainingInput
from sagemaker import image_uris
from sagemaker import hyperparameters

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[01/27/25 11:10:36] INFO     Found credentials from IAM Role:                                   ]8;id=104650;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=670220;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


### **Initializing session resource and client for acceding AWS using boto3**

boto3 provides you 3 of the impotent lyres to intact with the aes services
so they are
> clients

> resources 

> sessions

Lets start with understanding of **sessions**

sessions are mainly used when we what to intact with a perticular user in a perticular region or with some other credentials. so basically by using session we can have a connections to the services with respect to the credentials , users , regions etc.. we just need to specify all the required information such as AWS Access Key Token , region etc.. so that we are ready to connect.

[Sessions](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/session.html)

**Clients**
Basically clients can used with every service provided by the aws, it is a low level interface where we can perform all the api calls such as tearing on the instances in the ec2, monitoring or knowing the status of resources , etc.., it is a thread safe and we required to provide all information such that we can make all the api calls. ex:- listing out buckets, sending messages through sqs, turning on or tuning off resources etc..

[clients](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/clients.html)

**resources**
 we use resources when we what high level abstraction with the services such as manipulating s3 buckets, deleting or creating instances, manipulating dynamodb etc.. this is not thread sage so it is not used in the real tiime api calls only used when a session is declared and many of the services cant be accessed through resource. complex api operations. 

 [Resources](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/resources.html)



In [23]:
bucket_name="textdemand"
my_region=boto3.session.Session().region_name #session is all about your personal account details in our case we are working in us east-1
print(my_region)

us-east-1


In [8]:
#creating bucket in s3 using resources
s3=boto3.resource('s3')
if my_region=="us-east-1":
    s3.create_bucket(Bucket=bucket_name)
    print('s')

s


### **Dataset loading**

In [3]:
data=pd.read_csv("prod_.csv")
data.head()

,product,year,expiri duretion in years,month,no. of new products bought,no of products left for selling,no. of products sold,prodects left,no of products to be bought,demand %
0,colgate,2019,2.0,1,60,60,46,0,46.0,76.666667
1,colgate,2019,2.0,2,32,46,38,14,24.0,118.750000
2,colgate,2019,2.0,3,40,62,47,22,25.0,117.500000
3,colgate,2019,2.0,4,36,51,40,15,25.0,111.111111
4,colgate,2019,2.0,5,56,67,58,11,47.0,103.571429


Splitting the dataset and formatting to csv

In [5]:
from sklearn.model_selection import train_test_split
train_df,test_df= train_test_split(data, test_size = 0.175, random_state=0)

In [6]:
train_df.to_csv('train.csv', index=False, header=None)
test_df.to_csv('test.csv',index=False,header=None)

pushing data into s3 bucket

In [11]:
!aws s3 cp prod_.csv s3://textdemand/prodDemand/data/

upload: ./prod_.csv to s3://textdemand/prodDemand/data/prod_.csv  


In [17]:
!aws s3 cp train.csv s3://textdemand/prodDemand/data/
!aws s3 cp test.csv s3://textdemand/prodDemand/data/

upload: ./train.csv to s3://textdemand/prodDemand/data/train.csv
upload: ./test.csv to s3://textdemand/prodDemand/data/test.csv 


Loading execution role if you are working in local note book you need to declare it in 'env and load it
this helps in executing tasks into sagemaker

In [8]:
role_arn = sagemaker.get_execution_role()

### preprocessing dataset

In [9]:
arr=data['product'].unique()

In [10]:
import json
transformations={arr[i]:i+1 for i in range(len(arr))}
with open('transformations.json', 'w') as f:
    json.dump(transformations, f)

In [43]:
!aws s3 cp transformations.json s3://textdemand/prodDemand/data/

upload: ./transformations.json to s3://textdemand/prodDemand/data/transformations.json


In [11]:
data['product']=data['product'].map(transformations)

In [16]:
#preparing cleaned data set

data_clean={
            "sold":data["no. of products sold"], #This is a output variable , this is necessary while trining the model using sagemaker
            "products":data['product'],
            "year":data["year"],
            "month":data["month"]
           }

In [17]:
data_clean=pd.DataFrame(data_clean)

In [18]:
data_clean.to_csv("cleaned_data.csv")
!aws s3 cp cleaned_data.csv s3://textdemand/prodDemand/data/

upload: ./cleaned_data.csv to s3://textdemand/prodDemand/data/cleaned_data.csv


In [19]:
from sklearn.model_selection import train_test_split
train_df,test_df= train_test_split(data_clean, test_size = 0.175, random_state=0)

In [20]:
train_df.to_csv('train_clean.csv', index=False, header=None)
test_df.to_csv('test_clean.csv',index=False,header=None)

In [21]:
!aws s3 cp train_clean.csv s3://textdemand/prodDemand/data/
!aws s3 cp test_clean.csv s3://textdemand/prodDemand/data/

upload: ./train_clean.csv to s3://textdemand/prodDemand/data/train_clean.csv
upload: ./test_clean.csv to s3://textdemand/prodDemand/data/test_clean.csv


In [25]:
bucket_name="textdemand"

In [28]:
!aws s3 ls s3://textdemand/prodDemand/ --recursive

2025-01-27 11:13:17       3631 prodDemand/data/cleaned_data.csv
2025-01-27 07:24:26      10506 prodDemand/data/prod_.csv
2025-01-27 07:53:41       1451 prodDemand/data/test.csv
2025-01-27 11:13:41        494 prodDemand/data/test_clean.csv
2025-01-27 07:53:40       6726 prodDemand/data/train.csv
2025-01-27 11:13:40       2325 prodDemand/data/train_clean.csv
2025-01-27 07:52:08        113 prodDemand/data/transformations.json


In [29]:
train_df

,sold,products,year,month
97,300,3,2020,2
143,150,6,2020,12
152,112,1,2021,9
106,350,3,2020,11
201,117,5,2021,10
...,...,...,...,...
67,106,6,2019,8
192,95,5,2021,1
117,55,4,2020,10
47,58,4,2019,12


In [30]:
training_data='train_clean.csv'
testing_data='test_clean.csv'
proj_name="prodDemand"
data_file="data"
training_path=f"s3://{bucket_name}/{proj_name}/{data_file}/{training_data}"
testing_path=f"s3://{bucket_name}/{proj_name}/{data_file}/{testing_data}"

In [31]:
print(training_path)

s3://textdemand/prodDemand/data/train_clean.csv


In [32]:
train_in=TrainingInput(training_path, content_type="csv")
test_in=TrainingInput(testing_path, content_type="csv")

In [33]:
print(test_in)

In [34]:
model_op = "s3://{}/{}/{}".format(bucket_name, proj_name, "model")
print(model_op)

s3://textdemand/prodDemand/model


In [48]:
model_img = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")
print(model_img)

[01/27/25 11:37:29] INFO     Ignoring unnecessary instance type: None.                            ]8;id=296687;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=112294;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#528\528]8;;\

811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest


In [ ]:
from sagemaker.sklearn.estimator import Sklearn

In [49]:
training_job="demandpred"
xgb_model = sagemaker.estimator.Estimator(
    image_uri=model_img,
    role=role_arn,
    base_job_name=training_job,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    output_path=model_op,
    sagemaker_session=sagemaker.Session(),
    volume_size=5,
)

In [50]:
xgb_model.set_hyperparameters(
    n_estimators=2, max_depth=2, learning_rate=1,num_round=10,objective="reg:linear"
)

In [51]:
job_name = training_job + datetime.today().strftime("%Y-%m-%d-%H-%M-%S")
print(job_name)

demandpred2025-01-27-11-37-39


In [52]:
xgb_model.fit({"train": train_in, "validation": test_in}, wait=True, job_name=job_name)

[01/27/25 11:37:42] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=618650;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=970120;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#90\90]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name: demandpred2025-01-27-11-37-39         ]8;id=460201;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=223583;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\

2025-01-27 11:37:47 Starting - Starting the training job......
.....01-27 11:38:30 Starting - Preparing the instances for training.
..25-01-27 11:39:30 Downloading - Downloading input data.
.....01-27 11:40:00 Downloading - Downloading the training image.
2025-01-27 11:41:16 Training - Training image download completed. Training in progress.
2025-01-27 11:41:16 Uploading - Uploading generated training modelArguments: train
[2025-01-27:11:41:11:INFO] Running standalone xgboost training.
[2025-01-27:11:41:11:INFO] File size need to be processed in the node: 0.0mb. Available memory size in the node: 8455.32mb
[2025-01-27:11:41:11:INFO] Determined delimiter of CSV input is ','
[11:41:11] S3DistributionType set as FullyReplicated
[11:41:11] 184x3 matrix with 552 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2025-01-27:11:41:11:INFO] Determined delimiter of CSV input is ','
[11:41:11] S3DistributionType set as FullyReplicated
[11:41:11] 40x3 matrix with 

In [54]:
!aws s3 ls {bucket_name}/{proj_name}/model/ 

                           PRE demandpred2025-01-27-11-23-41/
                           PRE demandpred2025-01-27-11-33-14/
                           PRE demandpred2025-01-27-11-37-39/


In [56]:
!aws s3 ls {bucket_name}/{proj_name}/model/{job_name}/

                           PRE debug-output/
                           PRE output/
                           PRE profiler-output/


In [57]:
from sagemaker.serializers import CSVSerializer

In [58]:
Xgb_predict=xgb_model.deploy( initial_instance_count=1,instance_type="ml.t2.medium", serializer=CSVSerializer() )

[01/27/25 11:48:40] INFO     Creating model with name: demandpred-2025-01-27-11-48-40-463           ]8;id=491786;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=611099;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

[01/27/25 11:48:41] INFO     Creating endpoint-config with name demandpred-2025-01-27-11-48-40-463  ]8;id=771857;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=103859;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#5889\5889]8;;\

                    INFO     Creating endpoint with name demandpred-2025-01-27-11-48-40-463         ]8;id=513819;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=834992;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4711\4711]8;;\

-----------!

In [59]:
Xgb_predict.predict("2,2021,2")

b'50.551422119140625'

In [62]:
endpoint_name = Xgb_predict.endpoint_name
print(endpoint_name)

demandpred-2025-01-27-11-48-40-463


In [63]:
sgmkr_runtime = boto3.client("runtime.sagemaker")

In [65]:
response = sgmkr_runtime.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=text
)
print(response)

{'ResponseMetadata': {'RequestId': 'b0f66314-6eee-4e66-b320-8f309024307c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'b0f66314-6eee-4e66-b320-8f309024307c', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Mon, 27 Jan 2025 11:59:19 GMT', 'content-type': 'text/csv; charset=utf-8', 'content-length': '18', 'connection': 'keep-alive'}, 'RetryAttempts': 0}, 'ContentType': 'text/csv; charset=utf-8', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x7f0f483c9ae0>}


In [67]:
response['Body'].read().decode()

'50.551422119140625'

In [64]:
text="2,2021,2"

In [91]:
obj = s3_client.get_object( Bucket=bucket_name, Key="prodDemand/data/test_clean.csv")
testDf = pd.read_csv(obj["Body"])


In [114]:
testDf

,356,3,2020,1
0,100,5,2021,12
1,377,3,2021,2
2,145,6,2020,5
3,51,4,2021,3
4,99,1,2021,7
5,47,4,2019,9
6,100,5,2021,7
7,97,6,2019,5
8,56,1,2019,9
9,290,3,2020,12


In [92]:
import pandas as pd
import io

In [126]:
csv_buffer = io.StringIO()
testDf.to_csv(csv_buffer,header=None, index=None)
text=csv_buffer.getvalue()
text

'5,2021,12\n3,2021,2\n6,2020,5\n4,2021,3\n1,2021,7\n4,2019,9\n5,2021,7\n6,2019,5\n1,2019,9\n3,2020,12\n4,2019,2\n1,2019,6\n1,2021,2\n6,2019,4\n4,2021,5\n6,2020,8\n2,2022,3\n2,2022,4\n2,2020,7\n4,2020,3\n1,2020,3\n5,2020,6\n1,2020,4\n1,2021,12\n2,2019,1\n2,2021,3\n2,2021,5\n5,2020,5\n6,2020,4\n5,2019,8\n3,2020,9\n2,2021,1\n3,2021,4\n2,2019,7\n2,2019,4\n2,2020,9\n1,2019,8\n1,2020,5\n2,2021,7\n'

In [127]:
response = sgmkr_runtime.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=text
)
response['Body'].read().decode()

'85.62113952636719,320.0539855957031,112.4276351928711,74.39828491210938,81.93253326416016,61.46337890625,85.62113952636719,101.94023132324219,57.01935577392578,302.9542541503906,61.46337890625,57.01935577392578,81.93253326416016,101.94023132324219,74.39828491210938,112.4276351928711,64.25947570800781,64.25947570800781,25.297542572021484,71.9507827758789,56.67866516113281,83.17363739013672,56.67866516113281,81.93253326416016,25.63823699951172,50.551422119140625,50.551422119140625,83.17363739013672,112.4276351928711,72.68623352050781,302.9542541503906,50.551422119140625,320.0539855957031,25.63823699951172,25.63823699951172,25.297542572021484,57.01935577392578,56.67866516113281,50.551422119140625'

In [74]:
s3_client=boto3.client("s3")

In [ ]:
sgmkr_clnt.delete_endpoint(EndpointName=endpoint_name)